<img src="data/images/lecture-notebook-header.png" />

# Classification & Regression II: Tree Ensembles

Tree ensemble methods, also known as ensemble learning or ensemble methods, are machine learning techniques that combine multiple decision trees to make more accurate predictions. Ensemble methods aim to improve the predictive performance and robustness of individual decision trees by leveraging the diversity and collective wisdom of multiple models. There are two primary types of tree ensemble methods:

* **Random Forests:** Random forests construct an ensemble of decision trees by training each tree on a random subset of the training data and a random subset of the input features. During the prediction phase, each tree in the ensemble independently makes its prediction, and the final prediction is determined by aggregating the individual predictions, often using majority voting (for classification) or averaging (for regression). Random forests help reduce overfitting and increase generalization by introducing randomness in the training process.

* **Gradient Boosting:*8 Gradient boosting is an iterative ensemble method that builds a sequence of decision trees, where each subsequent tree is trained to correct the mistakes made by the previous trees. The trees are built in a stage-wise manner, with each tree fitting the residual errors of the previous ensemble. The final prediction is obtained by summing the predictions of all the trees in the ensemble. Gradient boosting algorithms, such as XGBoost and LightGBM, employ gradient descent optimization techniques to minimize a loss function and improve predictive accuracy.

Ensemble methods offer several advantages in machine learning:

* **Improved Predictive Accuracy:** By combining multiple trees, ensemble methods often achieve higher predictive accuracy compared to individual decision trees, especially when the underlying data is complex and contains noisy or ambiguous patterns.

* **Robustness and Generalization:** Ensemble methods reduce the risk of overfitting and improve generalization by leveraging the collective decision-making of multiple models. They can handle outliers and noisy data more effectively by aggregating predictions from different perspectives.

* **Feature Importance:** Ensemble methods can provide insights into feature importance. By analyzing the contribution of each feature across the ensemble, it becomes possible to identify the most influential factors in the predictive process.

However, ensemble methods also have some considerations:

* **Increased Complexity:** Ensemble methods are generally more computationally intensive and complex compared to individual decision trees. They require training and combining multiple models, which can increase computational resources and training time.

* **Model Interpretability:** While decision trees are highly interpretable, ensemble methods are often considered less interpretable due to the combined predictions of multiple trees. The ensemble's collective decision-making process can be challenging to interpret and explain.

Despite these considerations, ensemble methods, particularly random forests and gradient boosting, have gained significant popularity in various machine learning tasks due to their enhanced predictive accuracy and robustness. They are widely used in practice and have been applied to domains such as classification, regression, feature selection, and anomaly detection.

## Setting up the Notebook

### Specify how Plots Get Rendered

In [ ]:
%matplotlib inline

### Make all Required Imports. 

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from tqdm import tqdm

from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import BaggingClassifier, RandomForestClassifier, AdaBoostClassifier, GradientBoostingClassifier
from sklearn.model_selection import train_test_split

from sklearn.metrics import f1_score, mean_squared_error

---

## Prepare Training & Test

We have already done these steps many times. so there's no need for any details. As Decision Trees do not require normalized data, there's also not much to do in terms of data preprocessing.

### Load Data

The dataset used throughout this notebook is directly taken from [Kaggle](https://www.kaggle.com/sulianova/cardiovascular-disease-dataset). This dataset contains 70,000 records of patient data. Each record contains a series of personal and health related information (e.g., age, weight, height, blood pressure, smoker/non-smoker, etc.) The last attribute `cardio` is binary (`1` or `0`) indicating whether the patient is suffering from a cardiovascular disease. As such, the common task here is to predict if an unknown person is suffering from a cardiovascular disease based on his or her own personal and health information.

First we use `pandas` to load the dataset file into a DataFrame.

In [ ]:
df = pd.read_csv('data/datasets/cardio/cardio_train.csv', sep=';')

# Drop "artificial" feature id
df.drop(columns=['id'], inplace=True)

# Show the first 5 columns
df.head()

### Generate Training and Test Data

We use the usual 80/20 split for creating the training and test data.

In [ ]:
# Convert data to numpy arrays
X = df[['age', 'gender', 'height', 'weight', 'ap_hi', 'ap_lo', 'cholesterol', 'gluc', 'smoke', 'alco', 'active']].to_numpy()
y = df[['cardio']].to_numpy().squeeze()

# Split dataset in to training and test set
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)

print("Size of training set: {}".format(len(X_train)))
print("Size of test: {}".format(len(X_test)))

---

## Basic Decision Tree Classifier

For comparison, we train an individual Decision tree for different values of `max_depth`; same as in the Decision Tree notebook just with different data.

In [ ]:
max_depth = 20

# Keep track of depth and f1 scores for plotting
ds, f1s = [], []

# Loop over all values for max_depth
for d in tqdm(range(1, max_depth+1)):
    ds.append(d)
    # Train Decision Tree classifier for current value of max_depth
    clf = DecisionTreeClassifier(max_depth=d, criterion='gini', random_state=10).fit(X_train, y_train)
    # Predict class labels for test set
    y_pred = clf.predict(X_test)
    # Calculate f1 score between predictions and ground truth
    f1 = f1_score(y_test, y_pred)
    f1s.append(f1)
    
print('A maximum depth of {} yields the best f1 score of {:.3f}'.format(ds[np.argmax(f1s)], np.max(f1s), ))        
    
# Plot the results (max_depth vs. f1.score)
plt.figure()
plt.plot(ds, f1s)
plt.show()

---

## Bagging Classifier

A bagging classifier, short for Bootstrap Aggregating classifier, is an ensemble learning method in machine learning. It combines the predictions of multiple classifiers trained on bootstrap samples (randomly sampled subsets of the training data) to make collective predictions. Bagging classifiers are primarily used for classification tasks. Here's how a bagging classifier works:

* **Bootstrap Sampling:** The training dataset is randomly sampled with replacement to create multiple bootstrap samples. Each bootstrap sample typically has the same size as the original dataset but may contain repeated instances and exclude some original instances.

* **Classifier Training:** A separate classifier, often a decision tree or any other base classifier, is trained on each bootstrap sample. These classifiers are trained independently and can be trained in parallel, making bagging classifiers computationally efficient.

* **Voting or Averaging:** During the prediction phase, each classifier independently makes predictions for new, unseen instances. The final prediction is obtained by aggregating the predictions of all the classifiers through majority voting (for classification) or averaging (for probabilistic predictions).

Bagging classifiers offer several advantages:

* **Reduced Variance:** By training multiple classifiers on different bootstrap samples, bagging reduces the variance of predictions. It helps alleviate overfitting and improves the model's ability to generalize to unseen data.

* **Improved Stability:** Bagging classifiers are less sensitive to variations in the training data. The ensemble approach helps reduce the impact of outliers or noisy instances, leading to more robust predictions.

* **Parallelizability:** Since each classifier in the ensemble is trained independently, bagging classifiers can leverage parallel computing capabilities and significantly speed up the training process.

* **Model Interpretability:** While decision trees themselves are interpretable, the bagging ensemble approach may slightly reduce the interpretability due to the combined predictions of multiple trees. However, interpretability can still be obtained by analyzing individual decision trees within the ensemble.

It's important to note that bagging is applicable to any base classifier, not just decision trees. Hence, the term "bagging classifier" refers to the overall ensemble method rather than a specific classifier. As such, `scikit-learn`'s [BaggingClassifier](https://scikit-learn.org/stable/modules/generated/sklearn.ensemble.BaggingClassifier.html) gets as input a `base_estimator` which is a Decision Tree in our case. Note that we now have 2 parameters: 
 * `max_depth` of Decision Tree base estimator
 * `n_estimators` as the number of models

(well, there are more parameters but we just focus on these 2 here)

Since you have not 2 parameters to tune, we can implement this as nested loop to go over all combinations.

In [ ]:
max_depth = 20

ds, ns, f1s = [], [], []

# Loop over all values for max_depth
for d in tqdm(range(1, max_depth+1)):
    for n in [10, 25, 50, 100]:
        ds.append(d)
        ns.append(n)
        # Train Decision Tree classifier for current value of max_depth
        base_estimator = DecisionTreeClassifier(max_depth=d, random_state=10)
        clf = BaggingClassifier(base_estimator=base_estimator, n_estimators=n, max_features=1.0).fit(X_train, y_train)
        # Predict class labels for test set
        y_pred = clf.predict(X_test)
        # Calculate f1 score between predictions and ground truth
        f1 = f1_score(y_test, y_pred)
        f1s.append(f1)


We can visualize the result, i.e., the f1 scores for each parameter combination using a 3d plot.

In [ ]:
fig = plt.figure()
ax = fig.add_subplot(111, projection='3d')
ax.set_xlabel(r'max_depth', fontsize=16)
ax.set_ylabel(r'n_estimators', fontsize=16)
ax.set_zlabel('f1 score', fontsize=16)
ax.view_init(20, 60)
surf = ax.plot_trisurf(ds, ns, f1s, cmap=plt.cm.coolwarm, linewidth=0, antialiased=False)
plt.tight_layout()
plt.show()

We can also extract the best f1-score and the parameter combination that resulted in the score.

In [ ]:
f1_max = np.max(f1s)

print('The hights f1 score across all runs: {:.3f}'.format(f1_max))

In principle, multiple parameter combinations may yield the best f1-score. So in the code cell below, we list them all.

In [ ]:
best_runs = np.where(f1s == f1_max)[0]

print('The following runs resulted in the hightest f1 score of {:.3f}'.format(f1_max))
for run in best_runs:
    print('* max_depth = {}, n_estimators = {}'.format(ds[run], ns[run]))

---

## Random Forest Classifier

A Random Forest classifier is an ensemble learning method that combines the power of decision trees and bagging. It is a popular machine learning algorithm used for classification tasks. Random Forests build a collection of decision trees and aggregate their predictions to make the final classification decision. Here's how a Random Forest classifier works:

* **Bootstrap Sampling:** Random Forests use bootstrap sampling, where multiple random subsets (bootstrap samples) of the training data are created by sampling with replacement. Each bootstrap sample is typically of the same size as the original dataset but may contain repeated instances and exclude some original instances.

* **Feature Sampling:** For each bootstrap sample, an individual decision tree is constructed using a random subset of features. Random Forests introduce additional randomness by considering only a subset of the available features at each split of the decision tree. This helps to decorrelate the trees and promote diversity within the ensemble.

* **Voting:** During the prediction phase, each decision tree in the Random Forest independently predicts the class label for new, unseen instances. The final prediction is determined through majority voting, where the class label that receives the most votes from the decision trees is selected as the final prediction.

Random Forest classifiers offer several advantages:

* **Ensemble of Decision Trees:** Random Forests leverage the power of decision trees, allowing them to handle both categorical and numerical features, automatically handle missing values, and capture non-linear relationships between features and class labels.

* **Reduction of Overfitting:** By constructing multiple decision trees on different bootstrap samples and using feature randomness, Random Forests reduce overfitting and improve generalization performance. The ensemble approach helps to mitigate the risk of individual trees memorizing the training data.

* **Robustness to Outliers and Noisy Data:** Random Forests are robust to outliers and noisy data as the ensemble averages out the impact of individual noisy instances and reduces the influence of outliers.

* **Feature Importance:** Random Forests provide a measure of feature importance, allowing users to assess the relevance and contribution of each feature in the classification process.

Random Forest classifiers are widely used in various machine learning applications due to their effectiveness, robustness, and interpretability. They are particularly well-suited for handling high-dimensional data, dealing with complex relationships, and achieving high predictive accuracy in both binary and multi-class classification problems.

In [ ]:
max_depth = 20

ds, ns, f1s = [], [], []

# Loop over all values for max_depth
for d in tqdm(range(1, max_depth+1)):
    for n in [10, 25, 50, 100]:
        ds.append(d)
        ns.append(n)
        # Train Decision Tree classifier for current value of max_depth
        clf = RandomForestClassifier(max_depth=d, criterion='gini', n_estimators=n).fit(X_train, y_train)
        # Predict class labels for test set
        y_pred = clf.predict(X_test)
        # Calculate f1 score between predictions and ground truth
        f1 = f1_score(y_test, y_pred)
        f1s.append(f1)

And we can plot it scores for different parameter combinations again...

In [ ]:
fig = plt.figure()
ax = fig.add_subplot(111, projection='3d')
ax.set_xlabel(r'max_depth', fontsize=16)
ax.set_ylabel(r'n_estimators', fontsize=16)
ax.set_zlabel('f1 score', fontsize=16)
ax.view_init(20, 120)
surf = ax.plot_trisurf(ds, ns, f1s, cmap=plt.cm.coolwarm, linewidth=0, antialiased=False)
plt.tight_layout()
plt.show()

...as well as extracting the best score and respective parameter values.

In [ ]:
f1_max = np.max(f1s)

print('The hights f1 score across all runs: {:.3f}'.format(f1_max))

In principle, multiple parameter combinations may yield the best f1-score. So in the code cell below, we list them all.

In [ ]:
best_runs = np.where(f1s == f1_max)[0]

print('The following runs resulted in the hightest f1 score of {:.3f}'.format(f1_max))
for run in best_runs:
    print('* max_depth = {}, n_estimators = {}'.format(ds[run], ns[run]))

---

## AdaBoost Classifier

AdaBoost, short for Adaptive Boosting, is a popular ensemble learning algorithm in machine learning. It is primarily used for classification tasks, but it can also be applied to regression problems. AdaBoost combines the predictions of multiple weak classifiers (classifiers that perform slightly better than random guessing) to create a strong classifier with improved accuracy. AdaBoost is often used in combination with Decision Trees but are not limited to them. Here's how AdaBoost works:

* **Weighted Training Data:** Initially, each instance in the training dataset is assigned equal weight. The first weak classifier is trained on this weighted dataset.

* **Iterative Training:** In each iteration, the weights of misclassified instances from the previous iteration are increased, while the weights of correctly classified instances are decreased. This process focuses on difficult instances, allowing subsequent weak classifiers to pay more attention to them.

* **Classifier Weighting:** After each iteration, the performance of the weak classifier is evaluated, and a weight is assigned to it based on its accuracy. More accurate classifiers are given higher weights, indicating their importance in the final ensemble.

* **Classifier Combination:** The final prediction is obtained by combining the predictions of all weak classifiers, weighted by their respective classifier weights. The combined classifier has more emphasis on the predictions of the more accurate weak classifiers.

AdaBoost offers several advantages:

* **Improved Accuracy:** AdaBoost improves the overall classification accuracy by combining the predictions of multiple weak classifiers. The ensemble approach allows the algorithm to focus on challenging instances, making it particularly effective in handling complex classification problems.

* **Flexibility:** AdaBoost can work with various weak classifier algorithms, such as decision trees, logistic regression, or support vector machines. This flexibility allows it to adapt to different types of data and problem domains.

* **Robustness:** AdaBoost is robust to noise and outliers in the training data. The algorithm assigns higher weights to misclassified instances, allowing subsequent weak classifiers to concentrate on them and reduce the impact of outliers.

* **Interpretability:** While AdaBoost itself may not provide direct interpretability, it can be used in conjunction with interpretable weak classifiers. Decision trees, for example, can be employed as weak classifiers within AdaBoost, allowing for interpretability at the individual classifier level.

Despite its advantages, AdaBoost can be sensitive to noisy data and outliers. It is also computationally more expensive than training individual weak classifiers. However, AdaBoost remains a widely-used ensemble learning algorithm known for its ability to improve classification performance and handle complex datasets.

In [ ]:
max_depth = 10

ds, ns, f1s = [], [], []

# Loop over all values for max_depth
for d in tqdm(range(1, max_depth+1)):
    for n in [10, 25, 50, 100]:
        ds.append(d)
        ns.append(n)
        # Train Decision Tree classifier for current value of max_depth
        base_estimator = DecisionTreeClassifier(max_depth=d, random_state=100)
        clf = AdaBoostClassifier(base_estimator=base_estimator, n_estimators=n).fit(X_train, y_train)
        # Predict class labels for test set
        y_pred = clf.predict(X_test)
        # Calculate f1 score between predictions and ground truth
        f1 = f1_score(y_test, y_pred)
        f1s.append(f1)

And we can plot it scores for different parameter combinations again...

In [ ]:
fig = plt.figure()
ax = fig.add_subplot(111, projection='3d')
ax.set_xlabel(r'max_depth', fontsize=16)
ax.set_ylabel(r'n_estimators', fontsize=16)
ax.set_zlabel('f1 score', fontsize=16)
ax.view_init(20, 20)
surf = ax.plot_trisurf(ds, ns, f1s, cmap=plt.cm.coolwarm, linewidth=0, antialiased=False)
plt.tight_layout()
plt.show()

...as well as extracting the best score and respective parameter values.

In [ ]:
f1_max = np.max(f1s)

print('The hights f1 score across all runs: {:.3f}'.format(f1_max))

In principle, multiple parameter combinations may yield the best f1-score. So in the code cell below, we list them all.

In [ ]:
best_runs = np.where(f1s == f1_max)[0]

print('The following runs resulted in the hightest f1 score of {:.3f}'.format(f1_max))
for run in best_runs:
    print('* max_depth = {}, n_estimators = {}'.format(ds[run], ns[run]))

---

## Gradient Boosting Classifier

A Gradient Boosting classifier is a machine learning algorithm that combines the power of gradient boosting and decision trees for classification tasks. It is an ensemble learning method that builds a sequence of decision trees in an iterative manner, where each subsequent tree is trained to correct the mistakes made by the previous trees. Gradient Boosting classifiers are known for their high predictive accuracy and ability to handle complex patterns in the data. Here's how a Gradient Boosting classifier works:

* **Initialization:** Initially, the Gradient Boosting classifier starts with an initial prediction, which is usually a constant value or the mean of the target variable for regression tasks, or the class distribution for classification tasks.

* **Sequential Tree Building:** In each iteration, a decision tree is trained to fit the negative gradient (residuals) of the loss function with respect to the current ensemble's prediction. The tree is built to find the best splits that minimize the loss function, such as the mean squared error for regression or the log loss for classification.

* **Tree Weighting:** After each tree is trained, a weight or a learning rate is assigned to it based on the performance improvement it brings to the ensemble. The learning rate determines the contribution of each tree's prediction to the final prediction. Smaller learning rates typically yield more robust models, but at the cost of increased computation.

* **Ensemble Combination:** The final prediction of the Gradient Boosting classifier is obtained by summing up the predictions from all the trees, weighted by their respective weights or learning rates. The combination of multiple trees with adaptive weights leads to a strong classifier that is capable of capturing complex relationships in the data.

Gradient Boosting classifiers offer several advantages:

* **High Predictive Accuracy:** Gradient Boosting classifiers are known for their high predictive performance. By iteratively fitting the residuals, they focus on correcting the mistakes made by previous trees, leading to improved accuracy with each iteration.

* **Flexibility and Robustness:** Gradient Boosting classifiers can handle various types of data, including numerical and categorical features, as well as missing values. They are robust to outliers and noise in the data, as subsequent trees can learn to compensate for these anomalies.

* **Feature Importance:** Gradient Boosting classifiers can provide insights into feature importance. By analyzing the contribution of each feature across the ensemble, it becomes possible to identify the most influential factors in the classification process.

* **Interpretability:** While individual decision trees within a Gradient Boosting ensemble can be interpretable, the ensemble as a whole is generally less interpretable. However, interpretability can be gained by analyzing the individual trees and their contributions to the final prediction.

Gradient Boosting classifiers, such as XGBoost, LightGBM, and AdaBoost with gradient boosting, have gained significant popularity in various machine learning competitions and real-world applications. They are widely used for classification tasks where accuracy and robustness are critical.

In [ ]:
max_depth = 10

ds, ns, f1s = [], [], []

# Loop over all values for max_depth
for d in tqdm(range(1, max_depth+1)):
    for n in [10, 25, 50, 100]:
        ds.append(d)
        ns.append(n)    
        # Train Decision Tree classifier for current value of max_depth
        clf = GradientBoostingClassifier(max_depth=d, n_estimators=n).fit(X_train, y_train)
        # Predict class labels for test set
        y_pred = clf.predict(X_test)
        # Calculate f1 score between predictions and ground truth
        f1 = f1_score(y_test, y_pred)
        f1s.append(f1)

And we can plot it scores for different parameter combinations again...

In [ ]:
fig = plt.figure()
ax = fig.add_subplot(111, projection='3d')
ax.set_xlabel(r'max_depth', fontsize=16)
ax.set_ylabel(r'n_estimators', fontsize=16)
ax.set_zlabel('f1 score', fontsize=16)
ax.view_init(20, 150)
surf = ax.plot_trisurf(ds, ns, f1s, cmap=plt.cm.coolwarm, linewidth=0, antialiased=False)
plt.tight_layout()
plt.show()

...as well as extracting the best score and respective parameter values.

In [ ]:
f1_max = np.max(f1s)

print('The hights f1 score across all runs: {:.3f}'.format(f1_max))

In [ ]:
best_runs = np.where(f1s == f1_max)[0]

print('The following runs resulted in the hightest f1 score of {:.4f}'.format(f1_max))
for run in best_runs:
    print('* max_depth = {}, n_estimators = {}'.format(ds[run], ns[run]))

---

## Summary

Tree ensemble methods, also known as ensemble learning with decision trees, are powerful machine learning techniques that combine multiple decision trees to improve predictive accuracy and robustness. Ensemble methods aim to leverage the diversity and collective wisdom of multiple models to make more accurate predictions. Some popular tree ensemble methods include Random Forests, Gradient Boosting, and AdaBoost.

One of the key advantages of tree ensemble methods is their ability to achieve high predictive accuracy. By combining the predictions of multiple decision trees, ensemble methods can capture complex relationships and patterns in the data, leading to improved performance compared to individual trees. The ensemble approach helps mitigate overfitting and reduces the impact of noise and outliers, resulting in more robust predictions.

Another advantage is the flexibility of tree ensemble methods. They can handle both numerical and categorical features, as well as missing values, without requiring extensive data preprocessing. Ensemble methods are also suitable for various types of machine learning tasks, including classification, regression, feature selection, and anomaly detection. Additionally, tree ensemble methods can provide insights into feature importance, allowing users to identify the most influential factors driving the predictions.

However, tree ensemble methods also have some considerations. One limitation is the increased complexity compared to individual decision trees. Ensemble methods require training and combining multiple models, which can increase computational resources and training time. The interpretability of ensemble methods may be reduced compared to individual decision trees, as the combined predictions of multiple trees can be challenging to interpret and explain.

Furthermore, hyperparameter tuning can be crucial for tree ensemble methods to achieve optimal performance. Selecting the appropriate number of trees, tree depth, learning rate, and other hyperparameters can significantly impact the performance of the ensemble. Careful consideration and experimentation are required to find the right set of hyperparameters for a given problem.

In summary, tree ensemble methods offer high predictive accuracy, robustness to noise and outliers, and flexibility in handling various types of data. They provide insights into feature importance and can be applied to different machine learning tasks. However, ensemble methods come with increased complexity and may require careful hyperparameter tuning. Despite their limitations, tree ensemble methods have become widely used and are considered state-of-the-art approaches in machine learning due to their effectiveness and ability to tackle complex problems.